# ULMFiT

Universal Language Model Fine-tuning for Text Classification

Implemented with FastAI and Pytorch

Paper Reference: https://arxiv.org/abs/1801.06146
Code Source: https://github.com/fastai/course-nlp/blob/master/5-nn-imdb.ipynb

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.text import *
from scipy.spatial.distance import cosine as dist

In [ ]:
torch.cuda.device_count(), torch.cuda.get_device_name(0)

In [ ]:
# Define Batch Size. Language model can take a lot of space. Define samll batch size if you have small GPU. 
# bs=192
# bs=48
bs=24

In [ ]:
# Set GPU device
torch.cuda.set_device(0)

# 1. Prepare the IMDb data (on a sample)

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

It only contains one csv file, let's have a look at it.

It contains one line per review, with the label ('negative' or 'positive'), the text and a flag to determine if it should be part of the validation set or the training set. If we ignore this flag, we can create a DataBunch containing this data in one line of code:

### Load and preprocess the data and form a databunch
Add workaround for the bug in the fastai Text API

In [ ]:
%%time

# throws `BrokenProcessPool' Error sometimes. Keep trying `till it works!
count = 0
error = True
while error:
    try: 
        # Preprocessing steps
        data_lm = TextDataBunch.from_csv(path, 'texts.csv')
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')

A text is composed of words, and we can't apply mathematical functions to them directly. We first have to convert them to numbers. This is done in two differents steps: tokenization and numericalization. A TextDataBunch does all of that behind the scenes for you.


##### Tokenization

The first step of processing we make texts go through is to split the raw sentences into words, or more exactly tokens. The easiest way to do this would be to split the string on spaces, but we can be smarter:

    1. we need to take care of punctuation
    2. some words are contractions of two different words, like isn't or don't
    3. we may need to clean some parts of our texts, if there's HTML code for instance

To see what the tokenizer had done behind the scenes, let's have a look at a few texts in a batch.

The texts are truncated at 100 tokens for more readability. We can see that it did more than just split on space and punctuation symbols:

    1. the "'s" are grouped together in one token
    2. the contractions are separated like his: "did", "n't"
    3. content has been cleaned for any HTML symbol and lower cased
    4. there are several special tokens (all those that begin by xx), to replace unkown tokens (see below) or to introduce different text fields (here we only have one).


##### Numericalization

Once we have extracted tokens from our texts, we convert to integers by creating a list of all the words used. We only keep the ones that appear at list twice with a maximum vocabulary size of 60,000 (by default) and replace the ones that don't make the cut by the unknown token UNK.

The correspondance from ids tokens is stored in the vocab attribute of our datasets, in a dictionary called itos (for int to string).

In [ ]:
#data_lm.vocab.itos[:10]

In [ ]:
#data_lm.train_ds

In [ ]:
#data_lm.train_ds[0][0].data[:10]

# 2. Transfer Learning

We are going to create an IMDb language model starting with the pretrained weights from the wikitext-103 language model.

Now let's grab the full IMDb dataset for what follows.

In [ ]:
path = untar_data(URLs.IMDB)
path.ls()

In [ ]:
(path/'train').ls()

The reviews are in a training and test set following an imagenet structure. The only difference is that there is an unsup folder in train that contains the unlabelled data.

### we'll use a model pretrained on a bigger dataset
We're not going to train a model that classifies the reviews from scratch. Like in computer vision, we'll use a model pretrained on a bigger dataset (a cleaned subset of wikipedia called wikitext-103). That model has been trained to guess what the next word, its input being all the previous words. It has a recurrent structure and a hidden state that is updated each time it sees a new word. This hidden state thus contains information about the sentence up to that point.

We are going to use that 'knowledge' of the English language to build our classifier, but first, like for computer vision, we need to fine-tune the pretrained model to our particular dataset. Because the English of the reviews left by people on IMDB isn't the same as the English of wikipedia, we'll need to adjust a little bit the parameters of our model. Plus there might be some words extremely common in that dataset that were barely present in wikipedia, and therefore might no be part of the vocabulary the model was trained on.


##### More about WikiText-103

We will be using the WikiText-103 dataset created by Stephen Merity (https://smerity.com/) to pre-train a language model.

To quote Stephen's post:

The WikiText language modeling dataset is a collection of over 100 million tokens extracted from the set of verified Good and Featured articles on Wikipedia. The dataset is available under the Creative Commons Attribution-ShareAlike License.

Compared to the preprocessed version of Penn Treebank (PTB), WikiText-2 is over 2 times larger and WikiText-103 is over 110 times larger. The WikiText dataset also features a far larger vocabulary and retains the original case, punctuation and numbers - all of which are removed in PTB. As it is composed of full articles, the dataset is well suited for models that can take advantage of long term dependencies.

Download wikitext-103 (https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip). Unzip it into the .fastai/data/ folder on your computer.

### 2A. Package the IMDb data into a language model databunch


This is where the unlabelled data is going to be useful to us, as we can use it to fine-tune our model. Let's create our data object with the data block API (this takes a few minutes).

We'll to use a special kind of TextDataBunch for the language model, that ignores the labels (that's why we put 0 everywhere), will shuffle the texts at each epoch before concatenating them all together (only for training; we don't shuffle for the validation set) and will send batches that read that text in order with targets that are the next word in the sentence.

Add a try-except wrapper as a workaround for the bug in the fastai Text API


In [ ]:
%%time

# throws `BrokenProcessPool` Error sometimes. Keep trying `till it works!
count = 0
error = True
while error:
    try: 
        # Preprocessing steps
        data_lm = (TextList.from_folder(path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['train', 'test', 'unsup']) 
           # notebook 3-logreg-nb-imbd used .split_by_folder instead of .filter_by_folder
            # and this took less time to run. Can we do the same here?
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1, seed=42))
           #We randomly split and keep 10% (10,000 reviews) for validation
            #.label_for_lm()           
           #We want to make a language model so we label accordingly
            #.databunch(bs=bs, num_workers=1))
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')

##### I got faster results when I do the last two steps in a separate cell:

In [ ]:
%%time

# throws `BrokenProcessPool' Error sometimes. Keep trying `till it works!
count = 0
error = True
while error:
    try: 
        # Preprocessing steps
        #     the next step is the bottleneck
        data_lm = (data_lm.label_for_lm()           
           #We want to make a language model so we label accordingly
            .databunch(bs=bs, num_workers=1))
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')

In [ ]:
len(data_lm.vocab.itos),len(data_lm.train_ds)

In [ ]:
data_lm.show_batch()

In [ ]:
# Save the databunch for next time.
data_lm.save()

In [ ]:
# Load the saved data
data_lm = load_data(path, bs=bs)

### 2B. The Transfer Learning step.
This is where the magic happens!

The AWD_LSTM object contains the pretrained weights and the neural net architecture of the wikitext-103 language model. These will be downloaded the first time you execute the following line, and stored in ~/.fastai/models/ (or elsewhere if you specified different paths in your config file).

We import these into the language_model_learner object for our IMDb language model as follows:

In [ ]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

##### Get the IMDb language model vocabulary

In [ ]:
vocab = data_lm.vocab

In [ ]:
vocab.stoi["stingray"]

In [ ]:
vocab.itos[vocab.stoi["stingray"]]

In [ ]:
vocab.itos[vocab.stoi["mobula"]]

In [ ]:
learn_lm.model[0]

In [ ]:
awd = learn_lm.model[0]

##### Get the IMDb language model encoder. Recall that the encoder translates tokens into numerical vectors in the space defined by the IMDb vocabulary.

In [ ]:
enc = learn_lm.model[0].encoder

In [ ]:
enc

In [ ]:
enc.weight.size()

In [ ]:
enc.weight[0][:10]

##### Difference in vocabulary between IMDB and Wikipedia language models

We are going to load wiki_itos (the index-to-string list) from the wikitext 103 language model. We will compare the vocabularies of wikitext-103 and IMDB. It is to be expected that the two sets have some different vocabulary words, and that is no problem for transfer learning!

In [ ]:
#wiki_itos = pickle.load(open(Config().model_path()/'wt103-1/itos_wt103.pkl', 'rb'))
wiki_itos = pickle.load(open(Config().model_path()/'wt103-fwd/itos_wt103.pkl', 'rb'))

In [ ]:
wiki_itos[:10]

In [ ]:
len(wiki_itos)

In [ ]:
len(vocab.itos)

In [ ]:
i, unks = 0, []
while len(unks) < 50:
    if data_lm.vocab.itos[i] not in wiki_itos: unks.append((i,data_lm.vocab.itos[i]))
    i += 1

In [ ]:
wiki_words = set(wiki_itos)

In [ ]:
imdb_words = set(vocab.itos)

In [ ]:
wiki_not_imbdb = wiki_words.difference(imdb_words)

In [ ]:
imdb_not_wiki = imdb_words.difference(wiki_words)

In [ ]:
wiki_not_imdb_list = []

for i in range(100):
    word = wiki_not_imbdb.pop()
    wiki_not_imdb_list.append(word)
    wiki_not_imbdb.add(word)

In [ ]:
wiki_not_imdb_list[:15]

In [ ]:
imdb_not_wiki_list = []

for i in range(100):
    word = imdb_not_wiki.pop()
    imdb_not_wiki_list.append(word)
    imdb_not_wiki.add(word)

In [ ]:
imdb_not_wiki_list[:15]

All words that appear in the IMDB vocab, but not the wikitext-103 vocab, will be initialized to the same random vector in a model. As the model trains, we will learn their weights.

In [ ]:
vocab.stoi["modernisation"]

In [ ]:
"modernisation" in wiki_words

In [ ]:
vocab.stoi["30-something"]

In [ ]:
"30-something" in wiki_words, "30-something" in imdb_words

In [ ]:
vocab.stoi["linklater"]

In [ ]:
"linklater" in wiki_words, "linklater" in imdb_words

In [ ]:
"house" in wiki_words, "house" in imdb_words

In [ ]:
torch.allclose(enc.weight[vocab.stoi["30-something"], :], 
            enc.weight[vocab.stoi["linklater"], :])

In [ ]:
torch.allclose(enc.weight[vocab.stoi["30-something"], :], 
            enc.weight[vocab.stoi["house"], :])

In [ ]:
new_word_vec = enc.weight[vocab.stoi["linklater"], :]

#### Generating fake movie review-like text before fine tuning IMDb language model

In [ ]:
TEXT = "The color of the sky is"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
TEXT = "I hated this movie"
N_WORDS = 30
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
#doc(LanguageLearner.predict)

Lowering the temperature will make the texts less randomized.

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.10) for _ in range(N_SENTENCES)))

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.10) for _ in range(N_SENTENCES)))

##### 2C. Training (fine-tuning) the IMDb language model
Starting with the wikitext-103 pretrained weights, we'll fine-tune the model to "learn" the structure in the "language" of IMDb movie reviews.

In [ ]:
# Choose an appropriate learning rate.
learn_lm.lr_find()

In [ ]:
learn_lm.recorder.plot(skip_end=15)

In [ ]:
lr = 1e-3
lr *= bs/48

#### The first step in fine-tuning is to train only the last layer of the model.

In [ ]:
# This takes about a half-hour on an NVIDIA RTX-2070 GPU
# Fit for one cicle
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

In [ ]:
# Since this is relatively slow to train, we will save our weights
learn_lm.save('fit_1')

In [ ]:
learn_lm.load('fit_1')

In [ ]:
#learn_lm.unfreeze()

In [ ]:
# Fit for two cicle
#learn_lm.fit_one_cycle(2, lr, moms=(0.8,0.7))

#### Save the fine-tuned language model and the encoder

We have to save not just the fine-tuned IMDb language model but also its encoder. The language model is the part that tries to guess the next word. The encoder is the part that's responsible for creating and updating the hidden state.

In the next part we will build a sentiment classifier for the IMDb movie reviews. To do this we will need the encoder from the IMDb language model that we built.

In [ ]:
learn_lm.save('fine_tuned')

In [ ]:
learn_lm.save_encoder('fine_tuned_enc')

#### Load the saved model and its encoder

In [ ]:
learn_lm.load('fine_tuned')

In [ ]:
enc = learn_lm.model[0].encoder

In [ ]:
torch.allclose(enc.weight[vocab.stoi["30-something"], :], 
            enc.weight[vocab.stoi["linklater"], :])

In [ ]:
torch.allclose(enc.weight[vocab.stoi["30-something"], :], new_word_vec)

#### Generate movie review-like text, with the fine-tuned IMDb language model

Compare these texts to the ones generated with the untrained IMDb model in part 2A. Do they seem qualitatively better?

How good is our fine-tuned IMDb language model? Well let's try to see what it predicts when given a phrase that might appear in a movie review.

In [ ]:
TEXT = "i liked this movie because"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
TEXT = "This movie was"
N_WORDS = 30
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
TEXT = "I hated this movie"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

### 3. Building an IMDb Sentiment Classifier

We'll now use transfer learning to create a classifier, again starting from the pretrained weights of the wikitext-103 language model. We'll also need the IMDb language model encoder that we saved previously.

#### 3A. Load and preprocess the data, and form a databunch

Using fastai's flexible API, we will now create a different kind of databunch object, one that is suitable for a classifier rather than a for language model (as we did in 2A). This time we'll keep the labels for the IMDb movie reviews data.

Add the try-except wrapper workaround for the bug in the fastai Text API

Here the batch size can be decreased to avoid a CUDA out of memory error; your hardware may be able to handle a larger batch, in which case training will likely be faster.

Again, this takes a bit of time.

In [ ]:
bs = 8

In [ ]:
%%time

# throws `BrokenProcessPool' Error sometimes. Keep trying `till it works!
#    the progress bar has to complete three consecutive steps. Why three? 
#    fails nearly 100 times, and doesn't respond to interrupt
count = 0
error = True
while error:
    try: 
        # Preprocessing steps
        data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos']))
             #label them all with their folders
             #.databunch(bs=bs, num_workers=1))
        error = False
        print(f'failure count is {count}\n')   
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')

#### Form the preprocessed data into a databunch

In [ ]:
data_clas = data_clas.databunch(bs=bs, num_workers=1)

#### Save the databunch (since it took so long to make) and load it

In [ ]:
data_clas.save('imdb_textlist_class')

#### Load the databunch

In [ ]:
data_clas = load_data(path, 'imdb_textlist_class', bs=bs, num_workers=1)

In [ ]:
data_clas.show_batch()

#### 3B. Create a model to classify the IMDb reviews, and load the encoder we saved before.

Freeze the weights for all but the last layer and find a good value for the learning rate.

In [ ]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.3)
learn_c.load_encoder('fine_tuned_enc')
learn_c.freeze()

In [ ]:
learn_c.lr_find()

In [ ]:
learn_c.recorder.plot()

#### 3C. Training and fine-tuning the IMDb sentiment classifier

Train for one cycle, save intermediate result

In [ ]:
learn_c.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

In [ ]:
learn_c.save('first_cls')

In [ ]:
learn_c.load('first_cls');

#### Unfreeze last two layers and train for one cycle, save intermediate result.

In [ ]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn_c.save('second_cls')

In [ ]:
learn_c.load('second_cls')

#### Unfreeze the last three layers, and train for one cycle, and save intermediate result.

At this point we've already beaten the 2017 (pre-transfer learning) state of the art!

In [ ]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn_c.save('third_cls')

In [ ]:
learn_c.load('third_cls')

#### Unfreeze all the layers, train for two cycles, and save the result.

Note: at this step I encountered a CUDA error: unspecified launch failure. This is a known (and unsolved) problem with PyTorch when using an LSTM. https://github.com/pytorch/pytorch/issues/27837

Nothing to do but try again... and it worked on the second try.

In [ ]:
learn_c.unfreeze()
learn_c.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

#### The state of the art for this dataset in 2017 was 94.1%, and we have crushed it!!!

#### Save the IMDb classifer model

In [ ]:
learn_c.save('clas')

##### Let's look at a few examples, just to check that the classifier is working as we think it should.

The three outputs of the model predition are the label (pos or neg) and the class probability estimates for neg and pos, which meausure the model's confidence in it's prediction. As we'd expect, the model is extremely confident that the first review is pos and quite confident that the second review is neg. So it passes the test with flying colors.

In [ ]:
learn_c.predict("I really loved that movie, it was awesome!")

In [ ]:
learn_c.predict("I didn't really love that movie, and I didn't think it was awesome.")

Now that we've built the model, here is the part where you get to have some fun!! Take the model for a spin, try out your own examples!!